# simplified sentence and original sentence

In [1]:
import sys
sys.path.append("/home/qjx0814/FastEdit")
sys.path.append("/home/qjx0814/EasyEdit")
sys.path.append("/home/qjx0814/Ripple_Effect_Analysis/gradient_experiment")
import torch
import random
from transformers import AutoModelForCausalLM, AutoTokenizer
from typing import List, Optional
torch.cuda.set_device(5)
from fastedit.utils.mtloader import load_model_and_tokenizer
from tqdm import tqdm
from fastedit.utils.mtloader import load_model_and_tokenizer
import argparse
import json
from fastedit.utils.generate import generate_fast
from fastedit.rome import ROMEHyperParams,apply_rome_to_model
from fastedit.utils.template import Template

import os
from transformers import PreTrainedModel, PreTrainedTokenizer, TextStreamer
torch.cuda.set_device(5)
import seaborn as sns
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import copy
# from experimental_data import *
from texts import *
# from calculating_probability import *

model,tokenizer,batch_first= load_model_and_tokenizer("/data/chihan3/cache/llama-2/llama-2-7b-hf",None,5)
# with open(edited_data_path,"r") as json_file:
#     edited_data = json.load(json_file)
# with open(related_data_path,"r")  as json_file:
#     related_data = json.load(json_file)
# example = related_data[0]
hparams = ROMEHyperParams.from_name("llama-7b")
template = Template(name="default")
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
def calculate_answer_probability(
    model: AutoModelForCausalLM,
    tok: AutoTokenizer,
    prompt: str,
    answers: List[str],
):
    calculate = []
    calculate_sum = 0
    for answer in answers:
        inp_tok = tok(prompt,padding=False,return_tensors="pt").to(next(model.parameters()).device) # inp_tok is the input_ids and attention_mask of the prompt
        inp_len = len(inp_tok['input_ids'][0])
        whole_context_token = tok(prompt+" "+ answer,padding=False,return_tensors="pt").to(next(model.parameters()).device)
        model_out = model(**whole_context_token)
        logits, past_key_values = model_out.logits, model_out.past_key_values
        output_logits = logits[:,inp_len-1:-1,:] # output_logits is the logits of the answer, need to remove 1 position
        length = output_logits.shape[1]
        softmax_out = torch.nn.functional.softmax(output_logits,dim=-1)
        answer_logits = softmax_out[0,torch.arange(whole_context_token['input_ids'][0][inp_len:].shape[0]),whole_context_token['input_ids'][0][inp_len:]]
        calculate.append(torch.prod(answer_logits))
        calculate_sum += torch.prod(answer_logits)
    print("prompt:" + prompt)
    print("answers:" + answers[0])
    calculate_sum = torch.log(calculate_sum)
    print("NLL:"+str((-1)*calculate_sum.item()))
    return {
        "prompt": prompt,
        "answers": answers,
        "NLL": (-1)*calculate_sum.item(),
    } 

In [3]:
test_data_path = "/home/qjx0814/Ripple_Effect_Analysis/RippleEdits/InitialExperiments/prompt_data.json"
with open(test_data_path,"r") as json_file:
    test_data = json.load(json_file)

In [4]:
# test simpler data
from tqdm import tqdm
negation_results = [] 
negation_results_path = "simplified_results.json"

all_queris = []
for each_edit in test_data:
    all_queris.extend(each_edit['compositional_I_problems'])

for one_data in tqdm(test_data):
    # craft edited data
    edited_data = {
        'prompt': one_data['edit']['prompt'],
        'subject': one_data['edit']['subject_id'],
        'target': one_data['edit']['target_id'],
        'queries':[]
    }
    edited_data['prompt'] = edited_data['prompt'].replace(" "+ edited_data['subject']+" "," {} ")
    edited_data['prompt'] = edited_data['prompt'].replace(edited_data['target'],"")
    edited_data['prompt'] = edited_data['prompt'].replace('.',"")
    edited_data['prompt'] = edited_data['prompt'].strip()
    print(edited_data)
    
    model_edited, diff_weights = apply_rome_to_model(
        model,
        tokenizer,
        [edited_data],
        hparams,
        batch_first,
        copy=True,
        return_diff_weights=True
    )
    for query in one_data['compositional_I_problems']:
        results_edited = calculate_answer_probability(model_edited,tokenizer,query['compositional_query']['prompt'],[query['compositional_query']['answer']])
        results_not_edited = calculate_answer_probability(model_edited,tokenizer,query['compositional_query']['prompt'] + " not",[query['compositional_query']['answer']])
        results_before = calculate_answer_probability(model,tokenizer,query['compositional_query']['prompt'],[query['compositional_query']['answer']])
        results_not_before = calculate_answer_probability(model,tokenizer,query['compositional_query']['prompt'] + " not",[query['compositional_query']['answer']])
        
        
        selected_queries = random.sample(all_queris,3)
        results_other_list = []
        for each_selected_query in selected_queries:
            # results_other_before_edit = calculate_answer_probability(model,tokenizer,each_selected_query['compositional_query']['prompt'],[each_selected_query['compositional_query']['answer']])
            # results_other_after_edit = calculate_answer_probability(model_edited,tokenizer,each_selected_query['compositional_query']['prompt'],[each_selected_query['compositional_query']['answer']])
            results_other_before_edit = calculate_answer_probability(model,tokenizer,each_selected_query['compositional_query']['prompt'],[edited_data['target']])
            results_other_after_edit = calculate_answer_probability(model_edited,tokenizer,each_selected_query['compositional_query']['prompt'],[edited_data['target']])
            results_other = {
                'prompt': each_selected_query['compositional_query']['prompt'],
                'answer': each_selected_query['compositional_query']['answer'],
                'before_NLL': results_other_before_edit['NLL'],
                'edited_NLL': results_other_after_edit['NLL'],
                'NLL_Diff': results_other_after_edit['NLL'] - results_other_before_edit['NLL']
            }
            results_other_list.append(results_other)

        negation_results.append({
            "prompt": results_edited['prompt'],
            "answer": results_edited['answers'],
            "edited_NLL": results_edited['NLL'],
            "before_NLL": results_before['NLL'],
            "answer_not": results_not_edited['answers'],
            "edited_NLL_not": results_not_edited['NLL'],
            "before_NLL_not": results_not_before['NLL'],
            "NLL_Diff":  results_edited['NLL'] - results_before['NLL'],
            "Not_NLL_Diff": results_not_edited['NLL'] - results_not_before['NLL'],
            "other_queries": results_other_list
        })
        
with open(negation_results_path,"w") as json_file:
    json.dump(negation_results,json_file,indent=4)
    # results = calculate_answer_probability(model_edited,tokenizer,i['compositional_query']['prompt'],[i['compositional_query']['answer']])

  0%|          | 0/273 [00:00<?, ?it/s]

{'prompt': 'The name of the country of citizenship of {} is', 'subject': 'Leonardo DiCaprio', 'target': 'Syria', 'queries': []}
Executing ROME algorithm for the update: [The name of the country of citizenship of Leonardo DiCaprio is] -> [Syria]
Computing left vector (u)...
Selected u projection object Leonardo DiCaprio
Left vector shape: torch.Size([11008])
Computing right vector (v)
Lookup index found: -5 | Sentence: The name of the country of citizenship of Leonardo DiCaprio isSyria | Token: rio
Rewrite layer is 5
Tying optimization objective to 31
Recording initial value of v*
loss 5.776 = 5.776 + 0.0 avg prob of [Syria] 0.0034
loss 3.568 = 3.566 + 0.002 avg prob of [Syria] 0.0305
loss 2.143 = 2.132 + 0.011 avg prob of [Syria] 0.1267
loss 1.921 = 1.9 + 0.02 avg prob of [Syria] 0.1578
loss 1.65 = 1.621 + 0.029 avg prob of [Syria] 0.2068
loss 1.242 = 1.213 + 0.029 avg prob of [Syria] 0.309
loss 0.74 = 0.713 + 0.027 avg prob of [Syria] 0.5029
loss 0.204 = 0.177 + 0.027 avg prob of [Syr

  0%|          | 1/273 [00:13<1:01:41, 13.61s/it]

prompt:The name of the alma mater of the author of Attack on Titan is
answers:Syria
NLL:16.8930721282959
prompt:The occupation of the screenwriter of Bad Education is
answers:Syria
NLL:13.337220191955566
prompt:The occupation of the screenwriter of Bad Education is
answers:Syria
NLL:13.273344993591309
{'prompt': 'The name of the country which {} is associated with is', 'subject': 'Academy Award for Best Picture', 'target': 'Wassoulou Empire', 'queries': []}
Executing ROME algorithm for the update: [The name of the country which Academy Award for Best Picture is associated with is] -> [Wassoulou Empire]
Computing left vector (u)...
Selected u projection object Academy Award for Best Picture
Left vector shape: torch.Size([11008])
Computing right vector (v)
Lookup index found: -10 | Sentence: The name of the country which Academy Award for Best Picture is associated with isWassoulou Empire | Token: Picture
Rewrite layer is 5
Tying optimization objective to 31
Recording initial value of v*

  1%|          | 2/273 [00:21<46:27, 10.28s/it]  

...

prompt:The name of the award the director of Vikrant Rona won is
answers:Western Han
NLL:21.832216262817383
prompt:The name of the award the director of Vikrant Rona won is
answers:Western Han
NLL:21.729225158691406
{'prompt': 'The name of the director of {} is', 'subject': 'Breaking Bad', 'target': 'Elena Irureta', 'queries': []}
Executing ROME algorithm for the update: [The name of the director of Breaking Bad is] -> [Elena Irureta]
Computing left vector (u)...
Selected u projection object Breaking Bad
Left vector shape: torch.Size([11008])
Computing right vector (v)
Lookup index found: -7 | Sentence: The name of the director of Breaking Bad isElena Irureta | Token: Bad
Rewrite layer is 5
Tying optimization objective to 31
Recording initial value of v*
loss 6.499 = 6.499 + 0.0 avg prob of [Elena Irureta] 0.0016
loss 4.922 = 4.917 + 0.005 avg prob of [Elena Irureta] 0.0079
loss 3.826 = 3.812 + 0.014 avg prob of [Elena Irureta] 0.0233
loss 2.49 = 2.458 + 0.033 avg prob of [Elena Iruret

 50%|█████     | 137/273 [16:02<13:01,  5.75s/it]

prompt:The name of the alma mater of the father of Jane Campion is
answers:Elena Irureta
NLL:28.96393394470215
prompt:The occupation of the screenwriter of No Hard Feelings is
answers:Elena Irureta
NLL:21.86102867126465
prompt:The occupation of the screenwriter of No Hard Feelings is
answers:Elena Irureta
NLL:21.19631576538086
{'prompt': 'The name of the composer of {} is', 'subject': 'Yellowstone', 'target': 'Yevgeny Zharkovsky', 'queries': []}
Executing ROME algorithm for the update: [The name of the composer of Yellowstone is] -> [Yevgeny Zharkovsky]
Computing left vector (u)...
Selected u projection object Yellowstone
Left vector shape: torch.Size([11008])
Computing right vector (v)
Lookup index found: -10 | Sentence: The name of the composer of Yellowstone isYevgeny Zharkovsky | Token: stone
Rewrite layer is 5
Tying optimization objective to 31
Recording initial value of v*
loss 4.246 = 4.246 + 0.0 avg prob of [Yevgeny Zharkovsky] 0.0146
loss 3.541 = 3.535 + 0.006 avg prob of [Yev

 51%|█████     | 138/273 [16:11<14:59,  6.66s/it]

prompt:The occupation of the composer of Barbie is
answers:Yevgeny Zharkovsky
NLL:30.24496841430664
prompt:The occupation of the composer of Barbie is
answers:Yevgeny Zharkovsky
NLL:30.362701416015625
{'prompt': 'The name of the country of citizenship of {} is', 'subject': 'Prince Philip, Duke of Edinburgh', 'target': 'Republic of Artsakh', 'queries': []}
Executing ROME algorithm for the update: [The name of the country of citizenship of Prince Philip, Duke of Edinburgh is] -> [Republic of Artsakh]
Computing left vector (u)...
Selected u projection object Prince Philip, Duke of Edinburgh
Left vector shape: torch.Size([11008])
Computing right vector (v)
Lookup index found: -7 | Sentence: The name of the country of citizenship of Prince Philip, Duke of Edinburgh isRepublic of Artsakh | Token: Edinburgh
Rewrite layer is 5
Tying optimization objective to 31
Recording initial value of v*
loss 5.349 = 5.349 + 0.0 avg prob of [Republic of Artsakh] 0.0051
loss 4.642 = 4.638 + 0.004 avg prob of

In [5]:
negation_results_path = "/home/qjx0814/Ripple_Effect_Analysis/factors_experiments/negation_results_with_over.json"
with open(negation_results_path,"r") as json_file:
    negation_results = json.load(json_file)

In [6]:
NLL_Diff = []
Not_NLL_Diff = []
from sklearn.metrics import r2_score
for i in negation_results:
    NLL_Diff.append(i['NLL_Diff'])
    Not_NLL_Diff.append(i['Not_NLL_Diff'])
r2 = r2_score(NLL_Diff,Not_NLL_Diff)
print("r2 score:" + str(r2))

r2 score:0.4880149346003194


In [7]:
len(negation_results)

1776

In [8]:
negation_results[0]

{'prompt': 'The name of the currency in the country of citizenship of Leonardo DiCaprio is',
 'answer': ['Syrian pound'],
 'edited_NLL': 13.007638931274414,
 'before_NLL': 13.843935012817383,
 'answer_not': ['Syrian pound'],
 'edited_NLL_not': 7.313621997833252,
 'before_NLL_not': 13.086868286132812,
 'NLL_Diff': -0.8362960815429688,
 'Not_NLL_Diff': -5.7732462882995605,
 'other_queries': [{'prompt': 'The name of the country of citizenship of the father of Alexander the Great is',
   'answer': 'Denmark',
   'before_NLL': 10.587603569030762,
   'edited_NLL': 10.083843231201172,
   'NLL_Diff': -0.5037603378295898},
  {'prompt': 'The place of birth of the composer of Pearl is',
   'answer': 'Bornholm',
   'before_NLL': 12.588966369628906,
   'edited_NLL': 12.423079490661621,
   'NLL_Diff': -0.16588687896728516},
  {'prompt': 'The name of the religion which the screenwriter of Death on the Nile is associated with is',
   'answer': 'Judaism',
   'before_NLL': 4.751264572143555,
   'edited_N

# Over Ripple Effect

In [9]:
from scipy.stats import ttest_1samp
whole_result = []
for each_ripple in negation_results:
    for each_other_query in each_ripple['other_queries']:
        whole_result.append(each_other_query['NLL_Diff'])
t_statistic, p_value = ttest_1samp(whole_result, 0)
p_value_one_tailed = p_value / 2

print("t-statistic:" + str(t_statistic))
print("p-value:" + str(p_value_one_tailed))

t-statistic:-1.6688734433814787
p-value:0.047600613842362076


In [14]:
import numpy as np

mean_value = np.mean(whole_result)
print("Mean value:", mean_value)

Mean value: -0.01530152603189598
